# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb51805fb20>
├── 'a' --> tensor([[ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624]])
└── 'x' --> <FastTreeValue 0x7fb51805fb50>
    └── 'c' --> tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                        [ 0.1106,  0.3415, -0.8004,  1.0509],
                        [-1.1365,  0.8590,  0.2151,  1.5539]])

In [4]:
t.a

tensor([[ 0.7846,  1.5794, -1.2714],
        [ 0.6814,  0.7661, -0.0624]])

In [5]:
%timeit t.a

61.6 ns ± 0.0394 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb51805fb20>
├── 'a' --> tensor([[-2.1860,  0.4351,  1.9959],
│                   [-0.0530,  0.3706, -0.1443]])
└── 'x' --> <FastTreeValue 0x7fb51805fb50>
    └── 'c' --> tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                        [ 0.1106,  0.3415, -0.8004,  1.0509],
                        [-1.1365,  0.8590,  0.2151,  1.5539]])

In [7]:
%timeit t.a = new_value

64.1 ns ± 0.0507 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7846,  1.5794, -1.2714],
               [ 0.6814,  0.7661, -0.0624]]),
    x: Batch(
           c: tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7846,  1.5794, -1.2714],
        [ 0.6814,  0.7661, -0.0624]])

In [11]:
%timeit b.a

55.4 ns ± 0.0323 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9303, -0.8564,  0.4900],
               [-1.2648, -0.9290,  0.6270]]),
    x: Batch(
           c: tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.197 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

869 ns ± 8.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 27 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb46ccdbca0>
├── 'a' --> tensor([[[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]],
│           
│                   [[ 0.7846,  1.5794, -1.2714],
│                    [ 0.6814,  0.7661, -0.0624]]])
└── 'x' --> <FastTreeValue 0x7fb46ccdb820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 31 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb47392fcd0>
├── 'a' --> tensor([[ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624],
│                   [ 0.7846,  1.5794, -1.2714],
│                   [ 0.6814,  0.7661, -0.0624]])
└── 'x' --> <FastTreeValue 0x7fb46cdd0310>
    └── 'c' --> tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                        [ 0.1106,  0.3415, -0.8004,  1.0509],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 72.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3597,  0.9236, -0.2982, -0.6607],
                       [ 0.1106,  0.3415, -0.8004,  1.0509],
                       [-1.1365,  0.8590,  0.2151,  1.5539]],
              
                      [[-1.3597,  0.9236, -0.2982, -0.6607],
                       [ 0.1106,  0.3415, -0.8004,  1.0509],
                       [-1.1365,  0.8590,  0.2151,  1.5539]],
              
                      [[-1.3597,  0.9236, -0.2982, -0.6607],
                       [ 0.1106,  0.3415, -0.8004,  1.0509],
                       [-1.1365,  0.8590,  0.2151,  1.5539]],
              
                      [[-1.3597,  0.9236, -0.2982, -0.6607],
                       [ 0.1106,  0.3415, -0.8004,  1.0509],
                       [-1.1365,  0.8590,  0.2151,  1.5539]],
              
                      [[-1.3597,  0.9236, -0.2982, -0.6607],
                       [ 0.1106,  0.3415, -0.8004,  1.0509],
                       [-1.1365,  0.8590,  0.2151,  1.5539]],

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 57.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539],
                      [-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539],
                      [-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539],
                      [-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539],
                      [-1.3597,  0.9236, -0.2982, -0.6607],
                      [ 0.1106,  0.3415, -0.8004,  1.0509],
                      [-1.1365,  0.8590,  0.2151,  1.5539],
                      [-1.3597,  0.9236, -0.2982, -0.6607],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
